In [1]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from torchvision import transforms
from memory_module import MDNRNN
from vision_module import VAE


In [2]:
%pip install gym[box2d]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 374 kB 4.7 MB/s 
     |████████████████████████████████| 1.8 MB 51.8 MB/s 
     |████████████████████████████████| 18.3 MB 147 kB/s 
  ERROR: Failed building wheel for box2d-py
  Running setup.py clean for box2d-py
Failed to build box2d-py
    Running setup.py install for box2d-py ... done
  DEPRECATION: box2d-py was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


In [3]:
env = gym.make('CarRacing-v2', continuous=True)

/usr/local/lib/python3.8/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.8/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [129]:
obs = env.reset()
obs.shape

(96, 96, 3)

In [130]:
transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Grayscale(),
            transforms.Resize(64),
            transforms.ToTensor()
            ])
obs = transform(obs)
obs.shape, type(obs)

(torch.Size([1, 64, 64]), torch.Tensor)

In [131]:
vae = VAE()
mem = MDNRNN()
contr = torch.nn.Sequential(
            torch.nn.Linear(256+32, 3),
            torch.nn.Tanh())


In [133]:
z, _, _ = vae.encode(torch.unsqueeze(obs, 0))
z.shape

torch.Size([1, 32])

In [135]:
h = mem.init_hidden(1)
h[0].shape, h[0].shape

(torch.Size([1, 1, 256]), torch.Size([1, 1, 256]))

In [137]:
act = torch.tensor([[0,0,0]])
z_act = torch.cat((z, act), -1)
z_act = torch.unsqueeze(z_act, 0)
z_act.shape

torch.Size([1, 1, 35])

In [141]:
y, h = mem(z_act, h)
h[0].shape

torch.Size([1, 1, 256])

In [145]:
y, h_2 = mem(z_act, h)
h_2[0].shape

IndexError: ignored

In [161]:
#prev_act = torch.tensor([[0,0,0]])
#h = mem.init_hidden(1)
print('shape_h_init: ', h[0].shape)
print('shape act init', act_tensor.shape)
z, _, _ = vae.encode(torch.unsqueeze(obs, 0))  # z_shape([1, 32])
temp = torch.unsqueeze( torch.cat( (z, act_tensor), -1 ), -2)
print(temp.shape)
_, h = mem(temp, h)
print('shape_h_pred: ', h[0].shape)
print('len_h: ' , len(h))
print('shape_c : ', c.shape)

act_tensor = contr(torch.cat((z, h[0].view(1, -1)), -1))
    

shape_h_init:  torch.Size([1, 1, 256])
shape act init torch.Size([1, 3])
torch.Size([1, 1, 35])
shape_h_pred:  torch.Size([1, 1, 256])
len_h:  2
shape_c :  torch.Size([1, 1, 256])


In [154]:
act_tensor = contr(input_c)

In [166]:
act_tensor.view(-1).detach().numpy().shape

(3,)

In [74]:
act_tensor.view((1, 3)).shape

torch.Size([1, 3])

In [167]:
env.step(act_tensor.view(-1).detach().numpy())

(array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8), 7.252941176470588, False, {})

In [206]:
class Policy(nn.Module):

    continuous = True

    def __init__(self, device=torch.device('cpu')):
        super(Policy, self).__init__()

        self.device = device
        self.vision = VAE()
        self.memory = MDNRNN()
        self.controller = torch.nn.Sequential(
            torch.nn.Linear(256+32, 3),
            torch.nn.Tanh())

        self.last_act = torch.tensor([[0., 0., 0.]])
        self.h = self.memory.init_hidden(1)

        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Grayscale(),
            transforms.Resize(64),
            transforms.ToTensor()
            ])

    def forward(self, x):
        z, _, _ = self.vision.encode(torch.unsqueeze(x, 0))  # z_shape([1, 32])
        temp = torch.unsqueeze( torch.cat( (z, self.last_act), -1 ), -2)
        _, self.h = self.memory(temp, self.h)
        return self.controller(torch.cat((z, self.h[0].view(1, -1)), -1))
    
    def act(self, state):
        #preprocessing
        state = self.transform(state)
        self.last_act = self.forward(state)
        return self.last_act.view(-1).detach().numpy()

In [207]:
agent = Policy()

In [187]:
obs = env.reset()

In [188]:
obs = agent.transform(obs)

In [221]:
agent.act(obs)

array([ 0.09359595, -0.20643686, -0.13394757], dtype=float32)